# Census Demographic API: Snowflake Data Upload

- #### Reference: 
    - API User Guide: https://www.census.gov/content/dam/Census/data/developers/api-user-guide/api-guide.pdf
    - Geo specific pull: https://www.census.gov/data/developers/data-sets/acs-5year.html
    - High level populations: https://api.census.gov/data/2019/pep/population.html
    - Population by granular demo (race, gender, age, etc): https://www.census.gov/data/developers/data-sets/popest-popproj/popest.html
        - Variables: https://api.census.gov/data/2019/pep/charage/variables.htmlhttps://api.census.gov/data/2019/pep/charage/variables.html
- #### Available APIs: https://www.census.gov/data/developers/data-sets.html
- #### Annotations: https://www.census.gov/data/developers/data-sets/acs-1year/notes-on-acs-estimate-and-annotation-values.html
- #### Metadata Mapping:  https://www.census.gov/data/developers/data-sets/popest-popproj/popest/popest-vars.Vintage_2019.html


In [16]:
import requests

# Census data pull by NAME, SEX, RACE, AGE, HISPANIC, DATA REFRESH DATE, POPULATION
# Get API key after creating free account w/ Census online

data_url = f'https://api.census.gov/data/2019/pep/charage?get=NAME,SEX,RACE,AGE,HISP,POP&for=state:*&DATE_CODE=12&key=<APIKEY>'
response=requests.get(data_url)
print(response.text[:200])

[["NAME","SEX","RACE","AGE","HISP","POP","DATE_CODE","state"],
["Alabama","0","0","0","0","56901","12","01"],
["Alabama","0","0","0","1","51761","12","01"],
["Alabama","0","0","0","2","5140","12","01"


In [19]:
import pandas as pd

data=response.json()
df=pd.DataFrame(data[1:], columns=data[0]).\
    rename(columns={"NAME": "STATE", "SEX": "SEX", "RACE":"RACE", "AGE":"AGE", "HISP":"HISPANIC", \
                    "POP":"POPULATION", "DATE_CODE":"REFRESH_DT", "state":"STATE_ID"})
#df=df.astype(dtype={'Natural Increase':'int64','Net Domestic Mig':'int64','Net Foreign Mig':'int64'},inplace=True)
df.head()

,STATE,SEX,RACE,AGE,HISPANIC,POPULATION,REFRESH_DT,STATE_ID
0,Alabama,0,0,0,0,56901,12,01
1,Alabama,0,0,0,1,51761,12,01
2,Alabama,0,0,0,2,5140,12,01
3,Alabama,0,1,0,0,35594,12,01
4,Alabama,0,1,0,1,31393,12,01


In [20]:
len(df)

479502

In [30]:
df['REFRESH_DT'] = df['REFRESH_DT'].replace('12','2019')
df.head()

,STATE,SEX,RACE,AGE,HISPANIC,POPULATION,REFRESH_DT,STATE_ID
0,Alabama,0,0,0,0,56901,2019,01
1,Alabama,0,0,0,1,51761,2019,01
2,Alabama,0,0,0,2,5140,2019,01
3,Alabama,0,1,0,0,35594,2019,01
4,Alabama,0,1,0,1,31393,2019,01


In [35]:
df.to_csv('/Users/uremekn/Downloads/census_demo_pop_2019.csv', encoding='utf8', header=False, index=False, index_label=None)

In [42]:
import snowflake.connector

ctx = snowflake.connector.connect(
    user='<USER>',
    password='<PASSWORD>',
    account='<ACCOUNT>',
    warehouse='<WAREHOUSE>',
    database='<DATABASE>',
    schema='<SCHEMA>'
    )

In [34]:
ctx.cursor().execute(
       """CREATE TABLE IF NOT EXISTS PUBLIC.CENSUS_DEMO_POP_2019
       ("STATE" NVARCHAR(50),
        "SEX" INTEGER,
        "RACE" INTEGER,
        "AGE" INTEGER, 
        "HISPANIC" INTEGER,
        "POPULATION" INTEGER,
        "REFRESH_DT" NVARCHAR(10),
        "STATE_ID" NVARCHAR(10)) """)

In [ ]:
ctx.cursor().execute(""" truncate PUBLIC.CENSUS_DEMO_POP_2019 """)

In [44]:
ctx.cursor().execute("PUT 'file:///Users/uremekn/Downloads/census_demo_pop_2019.csv' @%CENSUS_DEMO_POP_2019")
ctx.cursor().execute("COPY INTO PUBLIC.CENSUS_DEMO_POP_2019 purge = True")

# on_error='continue'

cursor().close()
ctx.close()

In [45]:
import pandas as pd

sql = """ SELECT *
          FROM PUBLIC.CENSUS_DEMO_POP_2019
          LIMIT 100 """

test = pd.read_sql_query(sql, con=ctx)
test.head()

,STATE,SEX,RACE,AGE,HISPANIC,POPULATION,REFRESH_DT,STATE_ID
0,Alabama,0,0,0,0,56901,2019,01
1,Alabama,0,0,0,1,51761,2019,01
2,Alabama,0,0,0,2,5140,2019,01
3,Alabama,0,1,0,0,35594,2019,01
4,Alabama,0,1,0,1,31393,2019,01
